In [21]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from scipy.stats.stats import kendalltau
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D, Conv1DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LSTM, Masking
from tensorflow.keras.layers import MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import activations
sns.set()
from sklearn.model_selection import KFold
from sklearn import metrics

In [13]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *

data = np.empty(shape=[0, 222])

all_data = glob.glob('./drive/MyDrive/compsci/all_data.csv')

for j in all_data:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    data = np.append(data, csvrows, axis=0)

print(data.shape)

Loading  ./drive/MyDrive/compsci/all_data.csv
(151452, 222)


In [19]:
X = data[:,:-2]
y = data[:,-2]

inputs = X.reshape(X.shape + (1,1))
# targets = to_categorical(y)
targets = (y)

print(inputs.shape)
print(targets.shape)

(151452, 220, 1, 1)
(151452,)


In [15]:
# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=6, shuffle=False)

# K-fold Cross Validation model evaluation
fold_no = 1

verbose, epoch, batch_size = 1, 1, 256
for train, test in kfold.split(inputs, targets):
  targets = to_categorical(y)
  lstmmodel = Sequential()
  lstmmodel.add(LSTM(128, return_sequences=True, input_shape=(inputs[train].shape[1],1)))
  lstmmodel.add(LSTM(9, return_sequences=True))
  lstmmodel.add(MaxPooling1D(pool_size=2))
  lstmmodel.add(Flatten())
  lstmmodel.add(Dense(512, activation='relu'))
  lstmmodel.add(Dense(128, activation='relu'))    
  lstmmodel.add(Dense(32, activation='relu'))
  lstmmodel.add(Dense(9, activation='softmax'))
  lstmmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
  
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  modelhistory= lstmmodel.fit(inputs[:,:,:,0][train], targets[train], epochs=epoch, 
                             verbose=verbose, validation_split=0.2, 
                             batch_size = batch_size)
  print(fold_no)
  lstmpredictions = lstmmodel.predict(inputs[:,:,:,0][test], verbose=1) 
  lstm_predict=np.argmax(lstmpredictions,axis=1)
  lstm_actual_value=np.argmax(targets[test],axis=1)
  lstmmetrics = metrics.classification_report(lstm_actual_value, lstm_predict, digits=3)
  print(lstmmetrics, 'lstm metrics')
  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
395/395 [==============================] - 42s 97ms/step - loss: 0.3709 - accuracy: 0.9061 - val_loss: 0.8079 - val_accuracy: 0.7826
1
789/789 [==============================] - 20s 25ms/step
              precision    recall  f1-score   support

           1      0.972     0.876     0.922     19062
           2      0.965     0.972     0.968      1098
           3      0.568     0.992     0.723      1698
           4      0.301     0.734     0.427       811
           5      0.340     0.190     0.244       522
           6      0.618     0.453     0.523       139
           7      0.897     0.402     0.555       281
           8      0.971     0.902     0.935      1631

    accuracy                          0.863     25242
   macro avg      0.704     0.690     0.662     25242
weighted avg      0.907     0.863     0.875     25242
 lstm metrics
-----------------------------------------------

In [23]:
X = data[:,:-2]
y = data[:,-2]

inputs = X.reshape(X.shape + (1,1))
# targets = to_categorical(y)
targets = (y)

print(inputs.shape)
print(targets.shape)

(151452, 220, 1, 1)
(151452,)


In [24]:
# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=6, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

verbose, epoch, batch_size = 1, 1, 64
for train, test in kfold.split(inputs, targets):
  targets = to_categorical(y)
  # Define the model architecture
  cnnmodel = Sequential()
  cnnmodel.add(Conv1D(filters=128, kernel_size=16,padding='same', activation='relu',input_shape=(inputs[train].shape[1],1)))
  cnnmodel.add(BatchNormalization())
  cnnmodel.add(Conv1D(filters=32, kernel_size=16,padding='same', activation='relu'))
  cnnmodel.add(BatchNormalization())
  cnnmodel.add(Conv1D(filters=9, kernel_size=16,padding='same', activation='relu'))
  cnnmodel.add(MaxPooling1D(pool_size=2,padding='same'))
  cnnmodel.add(Flatten())
  cnnmodel.add(Dense(512, activation='relu'))
  cnnmodel.add(Dense(128, activation='relu'))
  cnnmodel.add(Dense(32, activation='relu'))
  cnnmodel.add(Dense(9, activation='softmax'))
  cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  modelhistory= cnnmodel.fit(inputs[:,:,:,0][train], targets[train], epochs=epoch, 
                             verbose=verbose, validation_split=0.2, 
                             batch_size = batch_size)
  print(fold_no)
  cnnpredictions = cnnmodel.predict(inputs[:,:,:,0][test], verbose=1) 
  cnn_predict=np.argmax(cnnpredictions,axis=1)
  cnn_actual_value=np.argmax(targets[test],axis=1)
  cnnmetrics = metrics.classification_report(cnn_actual_value, cnn_predict, digits=3)
  print(cnnmetrics, 'CNN metrics')
  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
1578/1578 [==============================] - 24s 15ms/step - loss: 0.1861 - accuracy: 0.9505 - val_loss: 0.5266 - val_accuracy: 0.8420
1
789/789 [==============================] - 2s 3ms/step
              precision    recall  f1-score   support

           1      0.977     0.975     0.976     19062
           2      0.871     0.993     0.928      1098
           3      0.953     0.961     0.957      1698
           4      0.716     0.869     0.785       811
           5      0.733     0.241     0.363       522
           6      0.877     0.460     0.604       139
           7      0.640     0.872     0.738       281
           8      0.987     0.999     0.993      1631

    accuracy                          0.954     25242
   macro avg      0.844     0.796     0.793     25242
weighted avg      0.954     0.954     0.950     25242
 CNN metrics
------------------------------------------------